# Advanced `min` and `max`

Beyond basics:
- `key=` for custom order (same idea as `sorted`)
- multi-key (tuples) to break ties
- `itemgetter` / `attrgetter` for speed & clarity
- argmin/argmax patterns (index of min/max)
- picking dict key with smallest/largest value
- empty iterable safety with `default=`
- None/NaN-safe keys
- streaming (generators) to avoid materializing
- computing min **and** max in one pass
- custom ranks (e.g., severity) via key mapping

In [1]:
from operator import itemgetter, attrgetter
from math import isnan
from dataclasses import dataclass
import math

## 1) Recap: natural vs custom key

In [2]:
data = [1, -2, 3, -4, 5, -6]

In [3]:
min(data), max(data)

(-6, 5)

In [4]:
min(data, key=abs), max(data, key=abs)

(1, -6)

## 2) Multi-key (tuple) keys for tie-breaking
Lexicographic comparison lets you specify secondary criteria.

In [5]:
pairs = [("bob", -3), ("amy", -3), ("amy", -7)]
# Smallest by |value|, tie-break by name alphabetically
min(pairs, key=lambda t: (abs(t[1]), t[0]))

('amy', -3)

Reverse just one component by transforming *that* part:

In [6]:
# Max by abs(value), but if tie pick lexicographically smallest name
max(pairs, key=lambda t: (abs(t[1]), -ord(t[0][0])))  # or use (abs, name) with min(...)

('amy', -7)

## 3) Dictionaries of records: `itemgetter` and tuple keys

In [7]:
quotes = [
    ('AACC', 6.05, 6.07, 6.03, 6.05, 65800),
    ('AAME', 1.70, 1.82, 1.70, 1.82, 4300),
    ('AMZN', 2044.30, 2053.00, 2017.66, 2042.76, 3000000),
]  # (symbol, open, high, low, close, volume)

In [8]:
# Min by open price:
min(quotes, key=itemgetter(1))

('AAME', 1.7, 1.82, 1.7, 1.82, 4300)

In [9]:
# Max by close, tie-breaker by volume descending (negate to flip):
max(quotes, key=lambda r: (r[4], -r[5]))

('AMZN', 2044.3, 2053.0, 2017.66, 2042.76, 3000000)

With dicts:

In [10]:
data = [
    {'date': '2020-04-09', 'symbol': 'AAPL', 'open': 268.70, 'high': 270.04, 'low': 264.70, 'close': 267.99},
    {'date': '2020-04-09', 'symbol': 'MSFT', 'open': 166.36, 'high': 167.37, 'low': 163.33, 'close': 165.14},
    {'date': '2020-04-09', 'symbol': 'AMZN', 'open': 2044.30, 'high': 2053.00, 'low': 2017.66, 'close': 2042.76},
    {'date': '2020-04-09', 'symbol': 'FB',   'open': 175.90, 'high': 177.08, 'low': 171.57, 'close': 175.19}
]

In [11]:
min(data, key=itemgetter('low'))  # same as lambda d: d['low']

{'date': '2020-04-09',
 'symbol': 'MSFT',
 'open': 166.36,
 'high': 167.37,
 'low': 163.33,
 'close': 165.14}

## 4) Argmin / Argmax (index of min/max)
Use indices with a key on the underlying data.

In [12]:
arr = [5, -1, 7, -4, 2]
i_min = min(range(len(arr)), key=arr.__getitem__)
(i_min, arr[i_min])

(3, -4)

In [13]:
i_max = max(range(len(arr)), key=arr.__getitem__)
(i_max, arr[i_max])

(2, 7)

## 5) Dicts: key with smallest/largest value
Classic pattern for frequency tables, scores, distances, etc.

In [14]:
scores = {"a": 10, "b": 7, "c": 1}
k_min = min(scores, key=scores.get)
(k_min, scores[k_min])

('c', 1)

In [15]:
k_max = max(scores, key=scores.get)
(k_max, scores[k_max])

('a', 10)

## 6) Empty iterables: `default=`
Avoid `ValueError` by providing a fallback.

In [16]:
empty = []
min(empty, default=0)

0

In [17]:
max((x for x in empty if x % 2 == 0), default='N/A')  # works with generators too

'N/A'

## 7) None/NaN-safe keys
Place missing values last (or first) by keying on a tuple.

In [18]:
vals = [None, 3, 1, None, 2]
# Min ignoring None (push None to the end):
min(vals, key=lambda x: (x is None, x)) , max(vals, key=lambda x: (x is None, x))

(1, None)

In [19]:
nums = [float('nan'), 3.0, 1.0, float('nan')]
min(nums, key=lambda x: (isnan(x), x)), max(nums, key=lambda x: (isnan(x), x))

(1.0, nan)

## 8) Streaming: use generators to avoid building lists
Both `min`/`max` consume iterables lazily.

In [20]:
def numbers():
    for i in range(1, 1000):
        yield i

g = numbers()
min(g), max(numbers())  # note: g is consumed; create a new generator for max

(1, 999)

## 9) Compute min and max in **one pass**
`min(x)` + `max(x)` each traverse the data once. If you need both and the data is expensive to traverse, do a single scan.

In [21]:
def min_max(iterable, *, key=None, default=None):
    it = iter(iterable)
    try:
        first = next(it)
    except StopIteration:
        if default is not None:
            return default
        raise ValueError('min_max() arg is an empty iterable')
    k = (key or (lambda x: x))
    min_item = max_item = first
    min_key = max_key = k(first)
    for x in it:
        kx = k(x)
        if kx < min_key:
            min_key, min_item = kx, x
        if kx > max_key:
            max_key, max_item = kx, x
    return min_item, max_item

min_max(range(1, 1000))

(1, 999)

In [22]:
players = [{'name':'a','score':10},{'name':'b','score':7},{'name':'c','score':2}]
min_max(players, key=itemgetter('score'))

({'name': 'c', 'score': 2}, {'name': 'a', 'score': 10})

Provide a `default` for empty input:

In [23]:
min_max([], default='N/A')

'N/A'

## 10) Custom orders via a rank mapping
Map values to their rank, then use `key=rank.get`.

In [24]:
levels = ['DEBUG','INFO','WARN','ERROR','CRITICAL']
rank = {lvl:i for i,lvl in enumerate(levels)}
vals = ['INFO','ERROR','DEBUG','CRITICAL','WARN']
max(vals, key=rank.get), min(vals, key=rank.get)

('CRITICAL', 'DEBUG')

## 11) `attrgetter` with custom classes

In [25]:
@dataclass
class Person:
    name: str
    age: int
    def __repr__(self):
        return f"{self.name}({self.age})"

people = [Person('Bob', 25), Person('Ann', 33), Person('Zoe', 19)]
min(people, key=attrgetter('age')), max(people, key=attrgetter('name'))

(Zoe(19), Zoe(19))

## 12) Strings: case-insensitive `min`/`max`
Casefolding is Unicode-aware (better than lowercasing).

In [26]:
letters = ['Z','a','A','z','x','X']
min(letters, key=str.casefold), max(letters, key=str.casefold)

('a', 'Z')